# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [25]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0, encoding = "ISO-8859-1")
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,1/1/2018,Moving flood risk modelling forwards,Nature Climate Change,NaN,"Koks, E. (2018). Moving flood risk modelling f...",paper-title-number-1,NaN
1,1/1/2018,Economic Impacts of Irrigation Constrained Agr...,Water Economics & Policy,NaN,"Perez-Blanco, C. D., Koks, E. E., Calliari, E....",paper-title-number-1,NaN
2,1/1/2017,Adaptation to Sea Level Rise: A Multidisciplin...,Water Resources Research,NaN,"Scussolini, P., Tran, T. T. V., Koks, E., Diaz...",paper-title-number-2,NaN
3,1/1/2017,Household migration in disaster impact analysi...,Natural Hazards,NaN,"Husby, T. G., & Koks, E. E. (2017). Household ...",paper-title-number-3,NaN
4,1/1/2016,A multiregional impact assessment model for di...,Economic Systems Research,NaN,"Koks, E. E., & Thissen, M. (2016). A multiregi...",paper-title-number-4,NaN
5,1/1/2016,Regional disaster impact analysis: comparing i...,Natural Hazards & Earth System Sciences,NaN,"Koks, E. E., Carrera, L., Jonkeren, O., Aerts,...",paper-title-number-6,NaN
6,1/1/2015,Integrated direct and indirect flood risk mode...,Risk Analysis,NaN,"Koks, E. E., Bockarjova, M., de Moel, H., & Ae...",paper-title-number-7,NaN
7,1/1/2015,"Combining hazard, exposure and social vulnerab...",Environmental Science & Policy,NaN,"Koks, E. E., Jongman, B., Husby, T. G., & Botz...",paper-title-number-8,NaN
8,1/1/2014,Increasing flood exposure in the Netherlands: ...,Natural Hazards & Earth System Sciences,NaN,"Jongman, B., Koks, E. E., Husby, T. G., & Ward...",paper-title-number-9,NaN
9,1/1/2014,Effect of spatial adaptation measures on flood...,Regional Environmental Change,NaN,"Koks, E. E., de Moel, H., Aerts, J. C., & Bouw...",paper-title-number-10,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [26]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [27]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

UnicodeEncodeError: 'charmap' codec can't encode character '\x96' in position 432: character maps to <undefined>

In [28]:
md

'---\ntitle: "Regional disaster impact analysis: comparing input-output and computable general equilibrium models"\ncollection: publications\npermalink: /publication/1/1/2016-paper-title-number-6\ndate: 1/1/2016\nvenue: \'Natural Hazards &amp; Earth System Sciences\'\ncitation: \'Koks, E. E., Carrera, L., Jonkeren, O., Aerts, J. C., Husby, T. G., Thissen, M. &amp; Mysiak, J. (2016). Regional disaster impact analysis: comparing input\x96output and computable general equilibrium models.\xa0Natural Hazards and Earth System Sciences,\xa016(8), 1911-1924.\'\n---\nRecommended citation: Koks, E. E., Carrera, L., Jonkeren, O., Aerts, J. C., Husby, T. G., Thissen, M. & Mysiak, J. (2016). Regional disaster impact analysis: comparing input\x96output and computable general equilibrium models.\xa0Natural Hazards and Earth System Sciences,\xa016(8), 1911-1924.'

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
